In [78]:
package main
import (
  // std
    "fmt"
    "io/ioutil"
    "os"
  // web
	"encoding/json"
	"net/http"
  // aws
	"github.com/aws/aws-lambda-go/events"
	"github.com/aws/aws-lambda-go/lambda"
)

In [79]:
// remote weather api url
const apiUrl = "https://www.metaweather.com/api/location/"

// message we'll receive from the client
type Event struct {
	WoeID string `json:"woeid"`
}

// raw data from the remote weather api
type rawWeatherData struct {
	ConsolidatedWeather []struct {
		WeatherStateName string  `json:"weather_state_name"`
		MinTemp          float64 `json:"min_temp"`
		MaxTemp          float64 `json:"max_temp"`
		Humidity         int     `json:"humidity"`
		Predictability   int     `json:"predictability"`
	} `json:"consolidated_weather"`
	Title        string `json:"title"`
	LocationType string `json:"location_type"`
	Woeid        int    `json:"woeid"`
	LattLong     string `json:"latt_long"`
}

// message we will send to the client
type WeatherData struct {
	WeatherStateName string  `json:"weather_state_name"`
	MinTemp          float64 `json:"min_temp"`
	MaxTemp          float64 `json:"max_temp"`
	Title            string  `json:"title"`
	LattLong         string  `json:"latt_long"`
}

In [80]:
func returnHighestPredictability(data *rawWeatherData) (*WeatherData, error) {
	highestPredictability := 0
	var predictabilityIndex int

	for i, p := range data.ConsolidatedWeather {
		if p.Predictability > highestPredictability {
			highestPredictability = p.Predictability
			predictabilityIndex = i
		}
	}

	cleanedData := &WeatherData{
		WeatherStateName: data.ConsolidatedWeather[predictabilityIndex].WeatherStateName,
		MinTemp:          data.ConsolidatedWeather[predictabilityIndex].MinTemp,
		MaxTemp:          data.ConsolidatedWeather[predictabilityIndex].MaxTemp,
		Title:            data.Title,
		LattLong:         data.LattLong,
	}

	return cleanedData, nil
}

In [81]:
func GET(locId string) (string, error) {
// func HandleRequest(req events.APIGatewayProxyRequest) (events.APIGatewayProxyResponse, error) {
	var weather *WeatherData
	var event Event

// 	IMPORTANT! Set Cors headers
// 	resp := events.APIGatewayProxyResponse{Headers: make(map[string]string)}
// 	resp.Headers["Access-Control-Allow-Origin"] = "*"

// 	// This will create a log entry in Cloudwatch
// 	fmt.Println("Request body: " + req.Body)

// 	err := json.Unmarshal([]byte(req.Body), &event)
// 	if err != nil {
// 		resp.StatusCode = 500
// 		resp.Body = err.Error()
// 		return resp, nil
// 	}

// 	response, err := http.Get(apiUrl + event.WoeID)
	response, err := http.Get(apiUrl + locId)
// 	if err != nil {
// 		resp.StatusCode = 500
// 		resp.Body = err.Error()
// 		return resp, nil
// 	}
    
	body, err := ioutil.ReadAll(response.Body)
// 	if err != nil {
// 		resp.StatusCode = 500
// 		resp.Body = err.Error()
// 		return resp, nil
// 	}
	defer response.Body.Close()

	var data rawWeatherData
// 	if err := json.Unmarshal(body, &data); err != nil {
// 		resp.StatusCode = 500
// 		resp.Body = err.Error()
// 		return resp, nil
// 	}
    
    json.Unmarshal(body, &data)
    
	weather, err = returnHighestPredictability(&data)
// 	if err != nil {
// 		resp.StatusCode = 500
// 		resp.Body = err.Error()
// 		return resp, nil
    
// 	}

	re, err := json.Marshal(*weather)
// 	if err != nil {
// 		resp.StatusCode = 500
// 		resp.Body = err.Error()
// 		return resp, nil
// 	}
    fmt.Printf("%+v\n", weather)

// 	resp.StatusCode = 200
// 	resp.Body = string(re)
	
    return string(re), nil

}

In [82]:
func main() {
	// Start takes a handler and talks to an internal Lambda endpoint to pass requests to the handler
	//lambda.Start(HandleRequest)
    fmt.Print(GET("906057")[0])
}

// warning: expression returns 2 values, using only the first one: [string error]


In [83]:
main()

&{WeatherStateName:Light Rain MinTemp:11.995 MaxTemp:18.97 Title:Stockholm LattLong:59.332169,18.062429}
123

In [50]:
fmt.Printf("%+v\n", weather)

ERROR: repl.go:1:21: undefined identifier: weather

In [92]:
var weather *WeatherData
var event Event

response, err := http.Get(apiUrl + "906057")

body, err := ioutil.ReadAll(response.Body)
defer response.Body.Close()

var data rawWeatherData

json.Unmarshal(body, &data)

weather, err = returnHighestPredictability(&data)

re, err := json.Marshal(*weather)

In [98]:
fmt.Printf("%+v\n", string(re))

{"weather_state_name":"Light Rain","min_temp":11.995,"max_temp":18.97,"title":"Stockholm","latt_long":"59.332169,18.062429"}


125 <nil>

In [65]:
fmt.Printf("%+v\n", re)

[123 34 119 101 97 116 104 101 114 95 115 116 97 116 101 95 110 97 109 101 34 58 34 76 105 103 104 116 32 82 97 105 110 34 44 34 109 105 110 95 116 101 109 112 34 58 49 49 46 57 57 53 44 34 109 97 120 95 116 101 109 112 34 58 49 56 46 57 55 44 34 116 105 116 108 101 34 58 34 83 116 111 99 107 104 111 108 109 34 44 34 108 97 116 116 95 108 111 110 103 34 58 34 53 57 46 51 51 50 49 54 57 44 49 56 46 48 54 50 52 50 57 34 125]


427 <nil>

In [66]:
re

[123 34 119 101 97 116 104 101 114 95 115 116 97 116 101 95 110 97 109 101 34 58 34 76 105 103 104 116 32 82 97 105 110 34 44 34 109 105 110 95 116 101 109 112 34 58 49 49 46 57 57 53 44 34 109 97 120 95 116 101 109 112 34 58 49 56 46 57 55 44 34 116 105 116 108 101 34 58 34 83 116 111 99 107 104 111 108 109 34 44 34 108 97 116 116 95 108 111 110 103 34 58 34 53 57 46 51 51 50 49 54 57 44 49 56 46 48 54 50 52 50 57 34 125]

In [72]:
func main() {
	type ColorGroup struct {
		ID     int
		Name   string
		Colors []string
	}
	group := ColorGroup{
		ID:     1,
		Name:   "Reds",
		Colors: []string{"Crimson", "Red", "Ruby", "Maroon"},
	}
    fmt.Printf("%+v\n", group)
	b, err := json.Marshal(group)
	if err != nil {
		fmt.Println("error:", err)
	}
	os.Stdout.Write(b)
}

In [73]:
main()

{ID:1 Name:Reds Colors:[Crimson Red Ruby Maroon]}
{"ID":1,"Name":"Reds","Colors":["Crimson","Red","Ruby","Maroon"]}

In [ ]:
os.Stdout.Write()

In [109]:
a := []byte("woeid: 906057")

In [110]:
fmt.Print(a)

[119 111 101 105 100 58 32 57 48 54 48 53 55]

45 <nil>

In [105]:
xxconst convert string = "woeid: 906057"

In [106]:
fmt.Print(byte(convert))

ERROR: repl.go:1:16: cannot convert string to uint8: convert